In [1]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField
from langchain.runnables.hub import HubRunnable
from langchain_anthropic import ChatAnthropic
from langchain_community.utils.math import cosine_similarity

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [10]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini")

<h3>Using Runnable Lambda</h3>

In [11]:
prompt=PromptTemplate.from_template(template="""
    Given the user question below, classify it as either being about `Langchain`,`Anthropic`, or `Other`

    Do not respond with more than one word.

    <question>
    {question}
    </question>

    Classification:
""")

In [12]:
chain=prompt|llmGPT|StrOutputParser()

In [13]:
chain.invoke(input={"question":"What is Claude LLM?"})

'Anthropic'

In [14]:
langchainChain=PromptTemplate.from_template(
    template="""
        You are an expert in langchain.\
        Always answer questions starting with "As Prof. Langchain told me". \
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGPT|StrOutputParser()

In [15]:
anthropicChain=PromptTemplate.from_template(
    template="""
        You are an expert in anthropic.\
        Always answer questions starting with "As Prof. Anthropic told me". \
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGPT|StrOutputParser()

In [16]:
generalChain=PromptTemplate.from_template(
    template="""
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGPT|StrOutputParser()

In [17]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropicChain
    elif "langchain" in info["topic"].lower():
        return langchainChain
    else:
        return generalChain

In [18]:
finalChain=RunnableParallel(
    {"topic":chain,"question":RunnablePassthrough()}
)|RunnableLambda(func=route)|StrOutputParser()

In [19]:
finalChain.invoke(input="What is Claude?")

'As Prof. Anthropic told me, Claude is a family of AI language models developed by Anthropic. Named presumably after Claude Shannon, who is known as the father of information theory, these models are designed to understand and generate human-like text. Claude models are built with a focus on safety, interpretability, and user alignment, aiming to create AI systems that can assist users while minimizing harmful outputs and ensuring ethical usage.'

In [20]:
finalChain.invoke(input="What is ChatPromptTemplate?")

'As Prof. Langchain told me, a ChatPromptTemplate is a powerful tool used in the Langchain framework designed to facilitate the creation of prompts for conversational AI models. It allows developers to define templates for chat interactions by specifying different components like system prompts, user messages, and function calls in a structured way. This modular approach helps in easily adapting the conversational flow, enhancing user experiences, and maintaining consistency across various chat scenarios.'

In [21]:
print(finalChain.invoke(input="Who was Eklavya?"))

Eklavya was a character from the ancient Indian epic, the Mahabharata. He is often depicted as a talented archer and a dedicated practitioner of martial arts. Eklavya was the son of a tribal chief and belonged to a community that was not traditionally accepted in the societal hierarchy of that time.

Eklavya aspired to become a great archer and sought to learn from Drona, the royal guru who had taught the Pandavas and Kauravas. However, Drona refused to teach Eklavya due to his lower caste status. Determined to master archery, Eklavya created a clay statue of Drona and practiced diligently in front of it, eventually becoming an extremely skilled archer.

A notable story involving Eklavya is when he demonstrated his exceptional archery skills by shooting a dog’s bark to silence it, which caught the attention of Drona and his students. Drona acknowledged Eklavya's talent, but when Eklavya later approached Drona to seek recognition, Drona demanded his right thumb as "guru dakshina" (a tea

In [26]:
finalChain=RunnablePassthrough.assign(topic=itemgetter('question')|chain)|RunnableLambda(func=route)

In [27]:
finalChain.invoke(input={'question':'What is GPT?'})

'GPT, or Generative Pre-trained Transformer, is a type of artificial intelligence model developed by OpenAI that is designed for natural language processing tasks. It utilizes a deep learning architecture called the transformer, which allows it to understand and generate human-like text based on the input it receives. GPT models are pre-trained on a diverse range of internet text, enabling them to generate coherent and contextually relevant responses. They can be used for various applications, including conversational agents, content creation, language translation, and more.'

<h3> Routing by Symantic Similarity</h3>

In [28]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

In [29]:
math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [30]:
embeddings=SentenceTransformerEmbeddings()

C:\Users\MSI\AppData\Local\Temp\ipykernel_17072\481481593.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings()
C:\Users\MSI\AppData\Local\Temp\ipykernel_17072\481481593.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings()
c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthed

In [31]:
promptTemplates=[physics_template,math_template]
promptEmbeddings=embeddings.embed_documents(texts=promptTemplates)

In [32]:
len(promptEmbeddings),len(promptEmbeddings[0])

(2, 768)

In [33]:
# Illustrations
questions=["What's a Black Hole?","What's Integral Calculus?"]

In [35]:
cosine_similarity(X=embeddings.embed_documents(texts=questions),Y=promptEmbeddings)

array([[0.25714641, 0.18505648],
       [0.24262198, 0.27752085]])

In [36]:
cosine_similarity(X=embeddings.embed_documents(texts=questions),Y=promptEmbeddings).argmax(axis=0)

array([0, 1])

In [37]:
def promptRouter(input) -> ChatPromptTemplate:
    queryEmbedding=embeddings.embed_query(text=input['query'])
    similarity=cosine_similarity(X=[queryEmbedding],Y=promptEmbeddings)[0]
    mostSimilarTemplate=promptTemplates[similarity.argmax()]
    print("Using Math Template" if mostSimilarTemplate==math_template else "Using Physics Template")
    return PromptTemplate.from_template(template=mostSimilarTemplate)

In [38]:
chain={"query":RunnablePassthrough()}|RunnableLambda(func=promptRouter)|llmGPT|StrOutputParser()

In [39]:
chain.invoke(input="What's a Blackhole?")

Using Physics Template


'A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. This occurs when a massive star collapses under its own gravity at the end of its life cycle. The boundary surrounding a black hole is called the event horizon; once something crosses this boundary, it cannot return.\n\nBlack holes are characterized by three main properties: mass, charge, and angular momentum (rotation). There are different types of black holes: \n\n1. **Stellar black holes** form from the remnants of massive stars after they exhaust their nuclear fuel.\n2. **Supermassive black holes**, which exist at the centers of galaxies, can have millions to billions of times the mass of the sun and are thought to play a crucial role in galaxy formation and evolution.\n3. **Intermediate black holes**, whose formation is less understood, are between stellar and supermassive black holes in size.\n\nThe study of black holes helps scientists understand fundame

In [40]:
chain.invoke(input="What's Trigonometry?")

Using Math Template


'Trigonometry is a branch of mathematics that studies the relationships between the angles and sides of triangles, particularly right triangles. The term "trigonometry" is derived from the Greek words "trigonon" (triangle) and "metron" (measure). \n\n### Key Components of Trigonometry:\n\n1. **Basic Functions**: \n   - There are three primary trigonometric functions based on a right triangle:\n     - **Sine (sin)**: The ratio of the length of the opposite side to the length of the hypotenuse.\n     - **Cosine (cos)**: The ratio of the length of the adjacent side to the length of the hypotenuse.\n     - **Tangent (tan)**: The ratio of the length of the opposite side to the length of the adjacent side.\n   - These functions can be represented as:\n     - \\( \\sin(\\theta) = \\frac{\\text{opposite}}{\\text{hypotenuse}} \\)\n     - \\( \\cos(\\theta) = \\frac{\\text{adjacent}}{\\text{hypotenuse}} \\)\n     - \\( \\tan(\\theta) = \\frac{\\text{opposite}}{\\text{adjacent}} = \\frac{\\sin(\\

In [41]:
chain.get_graph().print_ascii()

+----------------------+   
| Parallel<query>Input |   
+----------------------+   
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *              
    +--------------+       
    | promptRouter |       
    +--------------+       
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
